# Speculative Decoding

SGLang now provides an EAGLE-based speculative decoding option. The implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

**Note:** Currently, Speculative Decoding in SGLang does not support radix cache.

### Performance Highlights

- Official EAGLE code ([SafeAILab/EAGLE](https://github.com/SafeAILab/EAGLE)): ~200 tokens/s
- Standard SGLang Decoding: ~156 tokens/s
- EAGLE Decoding in SGLang: ~297 tokens/s
- EAGLE Decoding in SGLang (w/ `torch.compile`): ~316 tokens/s

All benchmarks below were run on a single H100.

## EAGLE Decoding

To enable EAGLE-based speculative decoding, specify the draft model (`--speculative-draft-model-path`) and the relevant EAGLE parameters:

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-04 05:21:34] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=36909, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=812426474, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level

[2025-03-04 05:21:53 TP0] Init torch distributed begin.
[2025-03-04 05:21:53 TP0] Load weight begin. avail mem=60.64 GB


[2025-03-04 05:21:54 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-04 05:21:55 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.63it/s]


...

[2025-03-04 05:21:58 TP0] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
/public_sglang_ci/runner-c-gpu-1/_work/sglang/sglang/python/sglang/srt/model_loader/weight_utils.py:458: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full contr

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]

[2025-03-04 05:21:59 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=36.35 GB
[2025-03-04 05:21:59 TP0] KV Cache is allocated. K size: 0.16 GB, V size: 0.16 GB.
[2025-03-04 05:21:59 TP0] Memory pool end. avail mem=36.03 GB


[2025-03-04 05:21:59 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-03-04 05:22:00] INFO:     Started server process [1858170]
[2025-03-04 05:22:00] INFO:     Waiting for application startup.
[2025-03-04 05:22:00] INFO:     Application startup complete.
[2025-03-04 05:22:00] INFO:     Uvicorn running on http://127.0.0.1:36909 (Press CTRL+C to quit)


[2025-03-04 05:22:00] INFO:     127.0.0.1:35082 - "GET /v1/models HTTP/1.1" 200 OK
[2025-03-04 05:22:01] INFO:     127.0.0.1:42428 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-04 05:22:01 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-04 05:22:04] INFO:     127.0.0.1:42430 - "POST /generate HTTP/1.1" 200 OK
[2025-03-04 05:22:04] The server is fired up and ready to roll!


In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-04 05:22:05 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-04 05:22:06] INFO:     127.0.0.1:42446 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [3]:
terminate_process(server_process)

### EAGLE Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--cuda-graph-max-bs`:


In [4]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --cuda-graph-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-04 05:22:20] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=31354, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=467721768, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_requests_level=

[2025-03-04 05:22:38 TP0] Init torch distributed begin.
[2025-03-04 05:22:38 TP0] Load weight begin. avail mem=78.81 GB


[2025-03-04 05:22:38 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-04 05:22:39 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.66it/s]


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.36s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.36s/it]

[2025-03-04 05:22:44 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=38.91 GB
[2025-03-04 05:22:44 TP0] KV Cache is allocated. K size: 0.16 GB, V size: 0.16 GB.
[2025-03-04 05:22:44 TP0] Memory pool end. avail mem=38.59 GB


[2025-03-04 05:22:44 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-03-04 05:22:44] INFO:     Started server process [1860585]
[2025-03-04 05:22:44] INFO:     Waiting for application startup.
[2025-03-04 05:22:44] INFO:     Application startup complete.
[2025-03-04 05:22:44] INFO:     Uvicorn running on http://127.0.0.1:31354 (Press CTRL+C to quit)


[2025-03-04 05:22:45] INFO:     127.0.0.1:58334 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-04 05:22:45] INFO:     127.0.0.1:58344 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-04 05:22:45 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-04 05:22:49] INFO:     127.0.0.1:58348 - "POST /generate HTTP/1.1" 200 OK
[2025-03-04 05:22:49] The server is fired up and ready to roll!


In [5]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-04 05:22:50 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-04 05:22:50] INFO:     127.0.0.1:58358 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(server_process)

### EAGLE Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [7]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-04 05:23:04] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=37300, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=945310920, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False

[2025-03-04 05:23:09] Casting torch.bfloat16 to torch.float16.


[2025-03-04 05:23:22 TP0] Casting torch.bfloat16 to torch.float16.


[2025-03-04 05:23:23 TP0] Casting torch.bfloat16 to torch.float16.
[2025-03-04 05:23:23 TP0] Init torch distributed begin.


[2025-03-04 05:23:23 TP0] Load weight begin. avail mem=53.43 GB
[2025-03-04 05:23:23 TP0] The following error message 'operation scheduled before its operands' can be ignored.


...

[2025-03-04 05:23:41 TP0] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
/public_sglang_ci/runner-c-gpu-1/_work/sglang/sglang/python/sglang/srt/model_loader/weight_utils.py:458: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full contr

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.14s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.14s/it]

[2025-03-04 05:23:42 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=43.23 GB
[2025-03-04 05:23:42 TP0] KV Cache is allocated. K size: 0.04 GB, V size: 0.04 GB.
[2025-03-04 05:23:42 TP0] Memory pool end. avail mem=43.15 GB


[2025-03-04 05:23:42 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8192
[2025-03-04 05:23:43] INFO:     Started server process [1863245]
[2025-03-04 05:23:43] INFO:     Waiting for application startup.
[2025-03-04 05:23:43] INFO:     Application startup complete.
[2025-03-04 05:23:43] INFO:     Uvicorn running on http://127.0.0.1:37300 (Press CTRL+C to quit)


[2025-03-04 05:23:44] INFO:     127.0.0.1:40710 - "GET /v1/models HTTP/1.1" 200 OK
[2025-03-04 05:23:44] INFO:     127.0.0.1:40720 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-04 05:23:44 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-04 05:23:48] INFO:     127.0.0.1:40730 - "POST /generate HTTP/1.1" 200 OK
[2025-03-04 05:23:48] The server is fired up and ready to roll!


In [8]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-04 05:23:49 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-04 05:23:49] INFO:     127.0.0.1:40744 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [9]:
terminate_process(server_process)